__Nombre:__ Lucía Saiz Lapique

## 1. The World Bank's international debt data
<p>No es que los humanos solo tengamos deudas para administrar nuestras necesidades. Un país también puede endeudarse para administrar su economía. Por ejemplo, el gasto en infraestructura es un ingrediente costoso requerido para que los ciudadanos de un país lleven una vida cómoda. El Banco Mundial es la organización que proporciona deuda a los países.</p>

<!-- <p>En este notebook, vamos a analizar los datos de la deuda internacional recopilados por el Banco Mundial. El conjunto de datos contiene información sobre el monto de la deuda (en USD) que deben los países en desarrollo en varias categorías.</p>  -->
    

"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD
"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD
"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD
"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD
"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD
"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD
"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD
"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD
"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD
"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD
"PPG, official creditors (INT, current US$)",DT.INT.OFFT.CD
"Principal repayments on external debt, long-term (AMT, current US$)",DT.AMT.DLXF.CD
"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)",DT.INT.DPNG.CD
"PPG, bonds (AMT, current US$)",DT.AMT.PBND.CD
"PPG, bonds (INT, current US$)",DT.INT.PBND.CD
"PPG, commercial banks (AMT, current US$)",DT.AMT.PCBK.CD
"PPG, commercial banks (DIS, current US$)",DT.DIS.PCBK.CD
"PPG, commercial banks (INT, current US$)",DT.INT.PCBK.CD
"PPG, other private creditors (AMT, current US$)",DT.AMT.PROP.CD
"PPG, other private creditors (DIS, current US$)",DT.DIS.PROP.CD
"PPG, other private creditors (INT, current US$)",DT.INT.PROP.CD
"PPG, private creditors (AMT, current US$)",DT.AMT.PRVT.CD
"PPG, private creditors (DIS, current US$)",DT.DIS.PRVT.CD
"PPG, private creditors (INT, current US$)",DT.INT.PRVT.CD
"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",DT.AMT.DPNG.CD

<p>Vamos a encontrar las respuestas a preguntas como:

<p>¿Cuál es el monto total de la deuda que deben los países enumerados en el conjunto de datos?
<p>¿Qué país posee la cantidad máxima de deuda y cómo se ve esa cantidad?
<p>¿Cuál es el monto promedio de la deuda de los países a través de diferentes indicadores de deuda?
    
Además tenemos otro dataset en el que encontramos información histórica de algunos índices de desarrollo, entre los que se encuentran algunos de deuda como son:

Series Name,Series Code
"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN

"Central government debt, total (current LCU)",GC.DOD.TOTL.CN

"Central government debt, total (% of GDP)",GC.DOD.TOTL.GD.ZS

#### 1. Inicializar y cargar el contexto spark

In [250]:
from pyspark import SparkContext
sc = SparkContext("local", "First App") # inicializamos pyspark

In [251]:
data_file1 = "./indicadores_deuda2.csv" # cargamos base de datos modificada
deuda = sc.textFile(data_file1)

In [252]:
data_file2 = "./indicadores_desarrollo3.csv"# cargamos base de datos modificada
desarrollo = sc.textFile(data_file2)

Los dataset han sido alterados con excel ya que algunos nombres tenían comas y estaban estropeando el parseado. Tras el cambio, el nuevo delimitador es un punto y coma, como vemos con el take de ambos a continuacion. 

Los cambios ejecutados al dataset fueron para modificar el movimiento de las comas y el ajuste de los números para eliminar los puntos de los miles y poder convertirlos en float más adelante.

In [253]:
deuda.take(2)

['country_name;country_code;DT.AMT.BLAT.CD;DT.AMT.DLXF.CD;DT.AMT.DPNG.CD;DT.AMT.MLAT.CD;DT.AMT.OFFT.CD;DT.AMT.PBND.CD;DT.AMT.PCBK.CD;DT.AMT.PROP.CD;DT.AMT.PRVT.CD;DT.DIS.BLAT.CD;DT.DIS.DLXF.CD;DT.DIS.MLAT.CD;DT.DIS.OFFT.CD;DT.DIS.PCBK.CD;DT.DIS.PROP.CD;DT.DIS.PRVT.CD;DT.INT.BLAT.CD;DT.INT.DLXF.CD;DT.INT.DPNG.CD;DT.INT.MLAT.CD;DT.INT.OFFT.CD;DT.INT.PBND.CD;DT.INT.PCBK.CD;DT.INT.PROP.CD;DT.INT.PRVT.CD',
 'Afghanistan;AFG;61739336.9;100847181.9;;39107845;100847181.9;;;;;49114729.4;72894453.7;23779724.3;72894453.7;;;;39903620.1;53239440.1;;13335820;53239440.1;;;;']

In [254]:
desarrollo.take(2)

['country_name;country_code;anho;GC.DOD.TOTL.CN;GC.DOD.TOTL.GD.ZS;SP.DYN.CBRT.IN',
 'Albania;ALB;2014 [YR2014];1023040800000 ;733202469198429 ;12.259']

In [255]:
deuda = deuda.map(lambda x: x.split(';')).filter(lambda x: 'country_name' not in x) # parseamos los datos separando por comas y
# eliminamos la linea de titulo
deuda.take(2) 

[['Afghanistan',
  'AFG',
  '61739336.9',
  '100847181.9',
  '',
  '39107845',
  '100847181.9',
  '',
  '',
  '',
  '',
  '49114729.4',
  '72894453.7',
  '23779724.3',
  '72894453.7',
  '',
  '',
  '',
  '39903620.1',
  '53239440.1',
  '',
  '13335820',
  '53239440.1',
  '',
  '',
  '',
  ''],
 ['Albania',
  'ALB',
  '54250280.6',
  '790248675.2',
  '514185620',
  '182197616.7',
  '236447897.3',
  '0',
  '39445139.5',
  '170018.4',
  '39615157.9',
  '76050616.1',
  '317194512.5',
  '234321242.3',
  '310371858.4',
  '4542664.9',
  '2279989.2',
  '6822654.1',
  '13847333.6',
  '165602386.9',
  '87884000',
  '28101536.1',
  '41948869.7',
  '31030688.2',
  '4618504.3',
  '120324.7',
  '35769517.2']]

In [256]:
desarrollo = desarrollo.map(lambda x: x.split(';')).filter(lambda x: 'country_name' not in x) # parseamos los datos separando 
# por comas y eliminamos la linea de titulo
desarrollo.take(2)

[['Albania',
  'ALB',
  '2014 [YR2014]',
  '1023040800000 ',
  '733202469198429 ',
  '12.259'],
 ['Albania',
  'ALB',
  '2015 [YR2015]',
  '1145500940000 ',
  '798644448747259 ',
  '12.197']]

##### 2. Número de países distintos en cada dataset. Coinciden?

Para resolver el ejercicio, se hará un conteo de los distintos nombres que aparecen en ambos datasets para asegurar que no haya nombres repetidos y se comparará la cantidad que hay en cada uno.

In [257]:
paises_deuda = deuda.map(lambda x: x[0]).distinct().count()
paises_deuda

124

In [258]:
paises_desar = desarrollo.map(lambda x: x[0]).distinct().count()
paises_desar

51

In [259]:
print('En la primera base de datos (deuda) hay', paises_deuda, 'países y en la segunda (desarrollo) hay', paises_desar, 'países. No coinciden.')

En la primera base de datos (deuda) hay 124 países y en la segunda (desarrollo) hay 51 países. No coinciden.


##### 3. Total de deuda contraida por cada pais: total amount of debt (in USD) DT.AMT.MLAT.CD

Para ver la deuda que tiene cada pais, agrupamos los valores distintos de cada uno tras transformar los datos de la columna en cuestión a de tipo float (numérico y con decimales). El resultado es una lista de todos los países con su deuda. Debemos también eliminar los casos que no sean países (las Naciones Unidas por ejemplo, que también están incluídas).

In [260]:
prueba = deuda.map(lambda x: (x[0], float(x[5]))).reduceByKey(lambda x,y: x+y)
prueba.collect()

[('Mauritius', 74568335.5),
 ('Bolivia', 357721211.5),
 ('El Salvador', 298939687.1),
 ('Belize', 29683326.5),
 ('Jordan', 270498715.5),
 ('Madagascar', 60669367.3),
 ('Mozambique', 91225626.1),
 ('Costa Rica', 272808260.7),
 ('Georgia', 144062401.5),
 ('Belarus', 595438826.0),
 ('India', 4545609909.9),
 ('Zambia', 53068964.9),
 ('Azerbaijan', 513623656.5),
 ('Pakistan', 1578651453.9),
 ('Djibouti', 34254909.2),
 ('Tajikistan', 59062982.0),
 ('Togo', 44198800.7),
 ('Kenya', 391340889.9),
 ('Tunisia', 956604000.1),
 ('Burundi', 16115111.8),
 ('Angola', 124688691.9),
 ('Sri Lanka', 449771961.1),
 ('St. Lucia', 15019216.3),
 ('Ghana', 130202003.1),
 ('Serbia', 715223963.7),
 ('Russian Federation', 141669500.0),
 ('Moldova', 135842865.5),
 ('Malawi', 35659831.4),
 ('Sudan', 217194878.8),
 ('Gambia', 33231516.7),
 ('St. Vincent and the Grenadines', 21615600.7),
 ('Guyana', 34651806.5),
 ('Thailand', 39071571.5),
 ('Ukraine', 1072472830.3),
 ('Bosnia and Herzegovina', 290170946.5),
 ('Comoro

In [261]:
limitado = prueba.filter(lambda x: ('Federation' not in x[0]) and ('UN' not in x[0]) and ('IDA' not in x[0]))
limitado.collect()

[('Mauritius', 74568335.5),
 ('Bolivia', 357721211.5),
 ('El Salvador', 298939687.1),
 ('Belize', 29683326.5),
 ('Jordan', 270498715.5),
 ('Madagascar', 60669367.3),
 ('Mozambique', 91225626.1),
 ('Costa Rica', 272808260.7),
 ('Georgia', 144062401.5),
 ('Belarus', 595438826.0),
 ('India', 4545609909.9),
 ('Zambia', 53068964.9),
 ('Azerbaijan', 513623656.5),
 ('Pakistan', 1578651453.9),
 ('Djibouti', 34254909.2),
 ('Tajikistan', 59062982.0),
 ('Togo', 44198800.7),
 ('Kenya', 391340889.9),
 ('Tunisia', 956604000.1),
 ('Burundi', 16115111.8),
 ('Angola', 124688691.9),
 ('Sri Lanka', 449771961.1),
 ('St. Lucia', 15019216.3),
 ('Ghana', 130202003.1),
 ('Serbia', 715223963.7),
 ('Moldova', 135842865.5),
 ('Malawi', 35659831.4),
 ('Sudan', 217194878.8),
 ('Gambia', 33231516.7),
 ('St. Vincent and the Grenadines', 21615600.7),
 ('Guyana', 34651806.5),
 ('Thailand', 39071571.5),
 ('Ukraine', 1072472830.3),
 ('Bosnia and Herzegovina', 290170946.5),
 ('Comoros', 2182053.1),
 ('Colombia', 11009075

##### 4. Media de los indicadores de deuda (tabla uno): DT.AMT.BLAT.CD, DT.DIS.BLAT.CD, DT.INT.BLAT.CD

Para resolver este ejercicio, se coge por separado cada índice y se calcula la media sumando todos los valores de la columna en cuestión y dividiendo por la cantidad que haya en cada una. No incluímos los datos vacíos pues da error, en caso de tener en cuenta los huecos vacíos (que la media saldría diferente) habría que eliminarlo del count, ya que en la suma no influyen (son 0).

In [262]:
suma1 = deuda.filter(lambda x: '' not in x).map(lambda x: float(x[2])).sum()
contar1 = deuda.filter(lambda x: '' not in x).map(lambda x: float(x[2])).count()

In [263]:
media1 = round((suma1/contar1),2)

In [264]:
print('La media del indicador DT.AMT.BLAT.CD es', media1)

La media del indicador DT.AMT.BLAT.CD es 2970822362.39


In [265]:
suma2 = deuda.filter(lambda x: '' not in x).map(lambda x: float(x[11])).sum()
contar2 = deuda.filter(lambda x: '' not in x).map(lambda x: float(x[11])).count()

In [266]:
media2 = suma2/contar2

In [267]:
print('La media del indicador DT.DIS.BLAT.CD es', media2)

La media del indicador DT.DIS.BLAT.CD es 5410625976.1


In [268]:
suma3 = deuda.filter(lambda x: '' not in x).map(lambda x: float(x[18])).sum()
contar3 = deuda.filter(lambda x: '' not in x).map(lambda x: float(x[18])).count()

In [269]:
media3 = round((suma3/contar3), 2)

In [270]:
print('La media del indicador DT.INT.BLAT.CD es', media3)

La media del indicador DT.INT.BLAT.CD es 562152096.3


##### 5. Los 20 paises con DT.AMT.DLXF.CD más alto

Para resolver el ejercicio, como pide veinte países, debenoms eliminar las asociaciones (como en el ejercicio 3) y luego ordenar los datos del índice en cuestion de forma que devuelvan los datos más altos (orden contrario al ascendente). Después con un take, pedimos los 20 primeros de esa lista, es decir, los 20 mayores. 

In [271]:
paises = deuda.map(lambda x: (x[0], float(x[3]))).filter(lambda x: ('Federation' not in x[0]) and ('UN' not in x[0]) and ('IDA' not in x[0]))
paises.sortBy(lambda x: x[1], ascending = False).take(20)

[('China', 96218620835.7),
 ('Brazil', 90041840304.1),
 ('Turkey', 51555031005.8),
 ('South Asia', 48756295898.2),
 ('India', 31923507000.8),
 ('Indonesia', 30916112653.8),
 ('Kazakhstan', 27482093686.4),
 ('Mexico', 25218503927.0),
 ('Romania', 14013783350.4),
 ('Colombia', 11985674438.7),
 ('Angola', 11067045628.1),
 ('Cameroon', 10404814960.2),
 ('Venezuela', 9878659207.2),
 ('Egypt', 9692114176.9),
 ('Lebanon', 9506919669.6),
 ('South Africa', 9474257551.9),
 ('Vietnam', 8873505909.2),
 ('Pakistan', 8336013891.3),
 ('Ukraine', 8148995625.6),
 ('Philippines', 6479170917.7)]

##### 6. Pais con los datos informados todos los años.

En primer lugar, los datos informados de por si excluyen a todos los campos vacíos, así que filtramos la base de datos para que se nos eliminen las observaciones que tengan algún campo vacío. Lo siguiente a tener en cuenta son los años sobre los que la base de datos tiene información. Para ello, hacemos un distinct que nos devuelva los años distintos que hay y luego buscamos cuántos países distintos tienen datos de esos años. El resultado es que hay un solo país con datos en 2017, con lo cual se sobreentiende que ese será en país con todos los campos. Filtramos por países que tengan datos en 2017 y obtenemos la respuesta.

In [272]:
informados = desarrollo.filter(lambda x: '' not in x).map(lambda x: (x[0], x[2][0:4]))
informados.collect()

[('Albania', '2014'),
 ('Albania', '2015'),
 ('Albania', '2016'),
 ('Australia', '2014'),
 ('Australia', '2015'),
 ('Australia', '2016'),
 ('Bahamas', '2014'),
 ('Bahamas', '2015'),
 ('Bahamas', '2016'),
 ('Barbados', '2014'),
 ('Barbados', '2015'),
 ('Belarus', '2014'),
 ('Belarus', '2015'),
 ('Belarus', '2016'),
 ('Belize', '2014'),
 ('Bhutan', '2014'),
 ('Bhutan', '2015'),
 ('Bhutan', '2016'),
 ('Botswana', '2014'),
 ('Botswana', '2015'),
 ('Botswana', '2016'),
 ('Brazil', '2014'),
 ('Brazil', '2015'),
 ('Brazil', '2016'),
 ('Colombia', '2014'),
 ('Colombia', '2015'),
 ('Colombia', '2016'),
 ('El Salvador', '2014'),
 ('El Salvador', '2015'),
 ('El Salvador', '2016'),
 ('Georgia', '2014'),
 ('Georgia', '2015'),
 ('Georgia', '2016'),
 ('Guatemala', '2014'),
 ('Guatemala', '2015'),
 ('Guatemala', '2016'),
 ('Hungary', '2014'),
 ('Hungary', '2015'),
 ('Hungary', '2016'),
 ('Indonesia', '2014'),
 ('Indonesia', '2015'),
 ('Indonesia', '2016'),
 ('Ireland', '2014'),
 ('Ireland', '2015'),
 

In [273]:
informados.map(lambda x: x[1]).distinct().collect()

['2014', '2015', '2017', '2016']

In [274]:
informados.filter(lambda x: x[1] == '2014').count()

50

In [275]:
informados.filter(lambda x: x[1] == '2015').count()

44

In [276]:
informados.filter(lambda x: x[1] == '2016').count()

39

In [277]:
informados.filter(lambda x: x[1] == '2017').count()

1

In [278]:
informados2 = informados.filter(lambda x: x[1] == '2017')
pais = informados2.map(lambda x: x[0]).collect()

In [279]:
print('El único país que tiene datos en todos los campos todos los años es', pais)

El único país que tiene datos en todos los campos todos los años es ['Malawi']


##### 7. Media anual de los distintos indicadores de desarrollo

Las medias se calculan por separado, realizando una agrupación de los datos donde se sumen los campos del índice y el número de veces que aparece cada uno (una por país) de forma que la media resultante por año sea la división de uno entre el otro.

In [280]:
media_anual1 = desarrollo.map(lambda x: (x[2][0:5], (float(x[3]), 1))).reduceByKey(lambda x, y: (x[0]+y[0], x[1]+ y[1]))
media_anual1.map(lambda x: (x[0], x[1][0]/x[1][1])).collect()  # media del indicador GC.DOD.TOTL.CN en cada año

[('2017 ', 2239858619962.69),
 ('2015 ', 1.4691377091875821e+21),
 ('2014 ', 1.245870413323809e+21),
 ('2016 ', 2.8672938250729105e+20)]

In [281]:
media_anual2 = desarrollo.map(lambda x: (x[2][0:5], (float(x[4]), 1))).reduceByKey(lambda x, y: (x[0]+y[0], x[1]+ y[1]))
media_anual2.map(lambda x: (x[0], x[1][0]/x[1][1])).collect() # media del indicador GC.DOD.TOTL.GD.ZS en cada año

[('2017 ', 48659564238322.0),
 ('2015 ', 438817252250091.25),
 ('2014 ', 357259846506983.7),
 ('2016 ', 395892633259688.2)]

In [298]:
media_anual3 = desarrollo.map(lambda x: (x[2][0:5], (float(x[5]), 1))).reduceByKey(lambda x, y: (x[0]+y[0], x[1]+ y[1]))
media_anual3.map(lambda x: (x[0], round((x[1][0]/x[1][1]), 2))).collect() # media del indicador SP.DYN.CBRT.IN en cada año

[('2017 ', 34.59), ('2015 ', 16.8), ('2014 ', 17.58), ('2016 ', 16.14)]

##### 8. Podrías decirme el total de deuda acumulada DT.AMT.MLAT.CD por los 10 países con un valor en media menor de SP.DYN.CBRT.IN

Este ejercicio, que es más complicado, se debe llevar a cabo en dos partes. En primer lugar, calculamos la media para cada pais de los cuatro años juntos con la base de datos sobre el desarrollo, como hemos estado haciendo hasta el momento en los ejercicios anteriores. Los ordenamos de menor a mayor media (pues el ejercicio pide los 10 más bajos) y lo guardamos en un objeto.

La segunda parte del ejercicio, compara los datos de las medias calculadas con los de la primera base de datos (deudas). A través de un join (de tipo inner), unimos la base de datos de deudas con las columna que nos interesan del objeto anterior (el índice en cuestion y el nombre del país) y buscamos los diez con la media más baja. Por último, nos quedamos con el nombre del pais y su deuda asociada en base a los filtros ejecutados anteriormente.

In [295]:
media_pais = desarrollo.map(lambda x: (x[0], (float(x[5]), 1))).reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))
media_pais = media_pais.map(lambda x: (x[0], round((x[1][0]/x[1][1]), 2))).sortBy(lambda x: x[1])
media_pais.take(10)

[('Japan', 7.93),
 ('San Marino', 8.05),
 ('Korea', 8.37),
 ('Spain', 9.0),
 ('Hungary', 9.53),
 ('Singapore', 9.63),
 ('Switzerland', 10.47),
 ('Moldova', 10.6),
 ('Ukraine', 10.6),
 ('Barbados', 10.86)]

In [296]:
deudas_filtro = deuda.map(lambda x: (x[0], x[5]))
junto = media_pais.join(deudas_filtro)
junto.take(10)

[('El Salvador', (18.73, '298939687.1')),
 ('Belize', (22.07, '29683326.5')),
 ('Kyrgyz Republic', (27.03, '75278037.2')),
 ('Kazakhstan', (22.78, '1079948889.7')),
 ('Zambia', (38.3, '53068964.9')),
 ('Botswana', (26.42, '125652344.5')),
 ('Malawi', (35.56, '35659831.4')),
 ('Russian Federation', (13.17, '141669500')),
 ('Philippines', (22.93, '739056427.3')),
 ('Thailand', (10.92, '39071571.5'))]

In [297]:
resultado = junto.map(lambda x: (x[0], x[1][1]))
resultado.take(10)

[('El Salvador', '298939687.1'),
 ('Belize', '29683326.5'),
 ('Kyrgyz Republic', '75278037.2'),
 ('Kazakhstan', '1079948889.7'),
 ('Zambia', '53068964.9'),
 ('Botswana', '125652344.5'),
 ('Malawi', '35659831.4'),
 ('Russian Federation', '141669500'),
 ('Philippines', '739056427.3'),
 ('Thailand', '39071571.5')]

##### 9. ¿Hay alguna relación entre los nacimientos y el indicador DT.AMT.DLXF.CD? ¿Cómo podrías demostrarlo?

Para averiguar la posible relación que existe entre ello, comparamos los dos índices uniendo las bases de datos y teniendo en cuenta solo las columnas en cuestión. Para ver si existe alguna relación clara entre ellos de forma fácil, decidimos ordenar los datos en base a uno de los dos índices y ver si son mayores o menores de la misma forma en ambas bases de datos, es decir, si cuando un indicador es muy alto para un país, también lo es su tasa de nacimiento. 

Lo verificamos también, viendo el país con la tasa e indicadores máximos y mínimos.

In [243]:
desarrollo.map(lambda x: x[5]).take(5)

['12.259', '12.197', '12.08', '13.2', '12.9']

In [244]:
deuda.map(lambda x: x[3]).take(5)

['100847181.9', '790248675.2', '171185188.1', '11067045628.1', '845630723.8']

In [289]:
desarrollo_red = desarrollo.map(lambda x: (x[1], float(x[5])))
deuda_red = deuda.map(lambda x: (x[1], float(x[3])))
union = deuda_red.join(desarrollo_red)
union.take(5)

[('KGZ', (816419045.1, 27.7)),
 ('KGZ', (816419045.1, 27.4)),
 ('KGZ', (816419045.1, 26.0)),
 ('BLZ', (105609924.9, 22.067)),
 ('IDN', (30916112653.8, 19.537))]

In [290]:
union.sortBy(lambda x: x[1][1]).collect()

[('UKR', (8148995625.6, 10.3)),
 ('MDA', (769232867.0, 10.445)),
 ('MDA', (769232867.0, 10.609)),
 ('UKR', (8148995625.6, 10.7)),
 ('MDA', (769232867.0, 10.759)),
 ('UKR', (8148995625.6, 10.8)),
 ('THA', (5914706998.3, 10.836)),
 ('THA', (5914706998.3, 10.999)),
 ('ALB', (790248675.2, 12.08)),
 ('ALB', (790248675.2, 12.197)),
 ('ALB', (790248675.2, 12.259)),
 ('BLR', (4640037884.2, 12.4)),
 ('BLR', (4640037884.2, 12.5)),
 ('BLR', (4640037884.2, 12.5)),
 ('LCA', (20332673.8, 12.502)),
 ('RUS', (66589761833.5, 12.9)),
 ('RUS', (66589761833.5, 13.3)),
 ('RUS', (66589761833.5, 13.3)),
 ('GEO', (1552116953.6, 13.925)),
 ('GEO', (1552116953.6, 14.078)),
 ('GEO', (1552116953.6, 14.165)),
 ('BRA', (90041840304.1, 14.307)),
 ('BRA', (90041840304.1, 14.472)),
 ('BRA', (90041840304.1, 14.624)),
 ('VCT', (26411052.2, 15.011)),
 ('COL', (11985674438.7, 15.308)),
 ('COL', (11985674438.7, 15.515)),
 ('COL', (11985674438.7, 15.725)),
 ('JAM', (559597431.4, 16.46)),
 ('TUR', (51555031005.8, 16.494)),
 

In [292]:
union.max(lambda x: x[1][0]), union.max(lambda x: x[1][1])

(('BRA', (90041840304.1, 14.624)), ('ZMB', (1217643471.9, 38.645)))

In [293]:
union.min(lambda x: x[1][0]), union.min(lambda x: x[1][1])

(('TON', (10369910.6, 25.756)), ('UKR', (8148995625.6, 10.3)))

Vemos que, con las pruebas ejecutadas a primera vista, los indicadores no están relacionadas porque, al ordenarlos de menor a mayor, vemos como según aumenta el indicador en cuestión, no lo hace la tasa de nacimientos. Además, los mínimos y máximos muestran mucha variabilidad en ambos casos, ya que no coinciden los países ni el máximo ni del mínimo.

In [247]:
sc.stop()